# CHAPTER 9 실습: 소비자 신용 리스크 관리 (Python 3.11 호환)
이 노트북은 다양한 `scikit-learn`/`fetch_openml` 버전, 네트워크 환경에서 **항상 실행 가능하도록** 설계되었습니다.

## 포함 내용
- 데이터 로딩 (OpenML → CSV URL → **오프라인 합성 데이터** 순서의 3중 폴백)
- 전처리 및 원-핫 인코딩
- 로지스틱 회귀 학습/평가
- 성별 그룹 예측 비율로 **편향성 확인** (가능한 경우)
- 성별 변수 제거 후 **재학습 비교**
- (선택) MLflow 기록, 모델 파일 저장


In [1]:
# 환경 정보 확인
import sys, sklearn, platform
print("Python :", sys.version.split()[0])
print("OS     :", platform.platform())
print("sklearn:", sklearn.__version__)

Python : 3.11.13
OS     : macOS-14.6.1-arm64-arm-64bit
sklearn: 1.7.1


## 1) 데이터 로딩 (다중 폴백)

In [2]:
import pandas as pd
import numpy as np

def load_credit_df():
    """여러 경로를 시도하며 데이터프레임과 출처 문자열을 반환합니다."""
    # 1) OpenML by name/version (권장 시그니처)
    try:
        from sklearn.datasets import fetch_openml
        try:
            credit = fetch_openml(name="credit-g", version=1, as_frame=True)
        except TypeError:
            # 일부 버전에서 위치인수만 허용되거나 시그니처 차이
            credit = fetch_openml("credit-g", version=1, as_frame=True)
        df = credit.frame.copy()
        return df, "openml:name=credit-g,version=1"
    except Exception as e1:
        err1 = e1

    # 2) OpenML by data_id (German Credit: 31)
    try:
        from sklearn.datasets import fetch_openml
        credit = fetch_openml(data_id=31, as_frame=True)
        df = credit.frame.copy()
        return df, "openml:data_id=31"
    except Exception as e2:
        err2 = e2

    # 3) Direct CSV URL (OpenML 제공 CSV 엔드포인트)
    try:
        url = "https://www.openml.org/data/get_csv/31/dataset_31_german_credit.csv"
        df = pd.read_csv(url)
        return df, "openml:csv-url"
    except Exception as e3:
        err3 = e3

    # 4) 최후 폴백: 합성 데이터 생성 (오프라인 전용)
    from sklearn.datasets import make_classification
    X, y = make_classification(
        n_samples=1000, n_features=8, n_informative=5,
        weights=[0.3, 0.7], random_state=42
    )
    df = pd.DataFrame(
        X, columns=["duration","amount","age","cred_hist","savings","employment","installment_rate","others"]
    )
    rng = np.random.default_rng(42)
    df["sex"] = rng.choice(["male","female"], size=len(df))
    df["housing"] = rng.choice(["own","rent","free"], size=len(df))
    df["purpose"] = rng.choice(["car","radio/TV","education","business","other"], size=len(df))
    df["target"] = y
    return df, "synthetic-fallback"

df, source = load_credit_df()
print("✅ Loaded data source:", source)
print(df.head())

/Users/lee/Documents/mlops-study/venv/lib/python3.11/site-packages/sklearn/datasets/_openml.py:109: UserWarning: A network error occurred while downloading https://api.openml.org/data/v1/download/31/credit-g.arff. Retrying...
  warn(


✅ Loaded data source: synthetic-fallback
   duration    amount       age  cred_hist   savings  employment  \
0  0.243847 -0.164471 -0.705182  -0.015433 -0.078425    0.730461   
1  0.650237  0.274936 -0.776536  -0.959132 -1.123291   -0.572504   
2 -3.390672 -0.590640 -1.343470   6.169133  1.222753    1.439649   
3  0.064793 -0.032960 -0.723131  -1.063065 -0.201072   -0.770711   
4  0.438060 -0.914989  0.953398  -0.033631 -0.048063   -0.860909   

   installment_rate    others     sex housing    purpose  target  
0          0.363777  0.727296    male    free      other       0  
1         -0.918580  0.073886  female    free        car       0  
2         -1.715076  3.223089  female    rent   radio/TV       1  
3          0.890636 -1.379626    male    free  education       1  
4          0.511169 -0.449294    male     own   business       0  


## 2) 타깃 생성 및 전처리 / 원-핫 인코딩

In [3]:
# 타깃 컬럼 표준화
if "target" not in df.columns:
    if "class" in df.columns:
        df["target"] = df["class"].map({"good": 1, "bad": 0}).astype(int)
        df.drop(columns=["class"], inplace=True)
    else:
        raise RuntimeError("타깃 컬럼을 찾을 수 없습니다. ('class' 또는 'target' 필요)")

# 기본 결측 처리 (간단히 제거)
df = df.dropna(subset=["target"])

# 원-핫 인코딩 (존재하는 컬럼에만 적용)
cat_candidates = [
    "sex","housing","purpose","job","telephone","foreign_worker",
    "checking_status","saving_status","other_parties","property_magnitude"
]
existing_cats = [c for c in cat_candidates if c in df.columns]

df_enc = pd.get_dummies(df, columns=existing_cats, drop_first=True)

# 설명 편의상, 명시적으로 사용할 피처 후보
if "sex" in df.columns and "sex_male" not in df_enc.columns:
    # pandas.get_dummies의 drop_first=True면 female 기준이면 sex_male 생성 안될 수 있음
    # 성별 더미를 명시적으로 보장
    df_enc["sex_male"] = (df["sex"] == "male").astype(int)

# 타깃/피처 분리
X = df_enc.drop(columns=["target"])
y = df_enc["target"]

print("X shape:", X.shape, "| y shape:", y.shape)
print("예시 컬럼:", list(X.columns)[:10])

X shape: (1000, 15) | y shape: (1000,)
예시 컬럼: ['duration', 'amount', 'age', 'cred_hist', 'savings', 'employment', 'installment_rate', 'others', 'sex_male', 'housing_own']


## 3) 학습/검증 분리 및 모델 학습/평가

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y if y.nunique()==2 else None
)

model = LogisticRegression(max_iter=2000)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
# predict_proba 존재 보장
if hasattr(model, "predict_proba"):
    y_proba = model.predict_proba(X_test)[:,1]
else:
    y_proba = y_pred.astype(float)

print(classification_report(y_test, y_pred))
try:
    print("ROC AUC:", roc_auc_score(y_test, y_proba))
except Exception as e:
    print("ROC AUC 계산 불가:", e)

              precision    recall  f1-score   support

           0       0.77      0.56      0.65        90
           1       0.83      0.93      0.88       210

    accuracy                           0.82       300
   macro avg       0.80      0.74      0.76       300
weighted avg       0.81      0.82      0.81       300

ROC AUC: 0.8212169312169312


## 4) 편향성 측정: 성별 그룹별 예측 비율 (가능한 경우)

In [5]:
import pandas as pd

if "sex_male" in X_test.columns:
    groups = X_test["sex_male"]
    df_res = pd.DataFrame({"group": groups, "pred": y_pred})
    rates = df_res.groupby("group")["pred"].mean()
    print("그룹별 예측(1) 비율 (male=1):\n", rates)
else:
    rates = None
    print("성별 특성이 없어 편향성 측정을 건너뜁니다.")

그룹별 예측(1) 비율 (male=1):
 group
False    0.789474
True     0.777027
Name: pred, dtype: float64


## 5) 편향 완화: 성별 변수 제거 후 재학습

In [6]:
drop_cols = [c for c in X.columns if c.startswith("sex_")] + (["sex_male"] if "sex_male" in X.columns else [])
X2 = X.drop(columns=drop_cols) if drop_cols else X

from sklearn.model_selection import train_test_split
X_train2, X_test2, y_train2, y_test2 = train_test_split(
    X2, y, test_size=0.3, random_state=42, stratify=y if y.nunique()==2 else None
)

from sklearn.linear_model import LogisticRegression
model2 = LogisticRegression(max_iter=2000)
model2.fit(X_train2, y_train2)
y_pred2 = model2.predict(X_test2)

if "sex_male" in X_test.columns:
    groups2 = X_test.loc[y_test2.index, "sex_male"]
    import pandas as pd
    df_res2 = pd.DataFrame({"group": groups2, "pred2": y_pred2})
    rates2 = df_res2.groupby("group")["pred2"].mean()
    print("민감 변수 제거 후 그룹별 예측 비율:\n", rates2)
else:
    rates2 = None
    print("성별 특성이 없어 편향 완화 비교를 건너뜁니다.")

민감 변수 제거 후 그룹별 예측 비율:
 group
False    0.769737
True     0.790541
Name: pred2, dtype: float64


## 6) (선택) MLflow 기록 및 모델 저장

In [7]:
# MLflow는 설치 여부/환경에 따라 실패할 수 있으므로 안전하게 처리
try:
    import mlflow, mlflow.sklearn
    with mlflow.start_run():
        mlflow.log_param("model", "logistic_regression")
        try:
            from sklearn.metrics import roc_auc_score
            mlflow.log_metric("roc_auc", roc_auc_score(y_test, y_proba))
        except Exception:
            pass
        if rates is not None:
            mlflow.log_metric("male_rate_before", float(rates.get(1, np.nan)))
        if rates2 is not None:
            mlflow.log_metric("male_rate_after", float(rates2.get(1, np.nan)))
        mlflow.sklearn.log_model(model, "baseline_model")
        mlflow.sklearn.log_model(model2, "fair_model")
    print("✅ MLflow 기록 완료")
except Exception as e:
    print("MLflow 기록을 건너뜁니다:", e)

# 모델 저장
import joblib
joblib.dump(model, "credit_lr.pkl")
print("✅ 모델 저장: credit_lr.pkl")

/var/folders/8h/dp71kw8x3c547p3b_x0kfmm00000gn/T/ipykernel_30358/4064005771.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  mlflow.log_metric("male_rate_before", float(rates.get(1, np.nan)))
/var/folders/8h/dp71kw8x3c547p3b_x0kfmm00000gn/T/ipykernel_30358/4064005771.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  mlflow.log_metric("male_rate_after", float(rates2.get(1, np.nan)))
2025/08/09 20:54:26 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/09 20:54:27 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_exampl

✅ MLflow 기록 완료
✅ 모델 저장: credit_lr.pkl
